In [29]:
from Data_Handler.DataReader import DataReader
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from impressions import Impressions
from hybrid import *
from tqdm import tqdm
from evaluator import evaluate
import pandas as pd
import numpy as np
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.DataIO import DataIO
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical
import os

In [ ]:
class Linear_Hybrid(BaseRecommender):
    RECOMMENDER_NAME = "Linear_Hybrid"

    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3, recommender_4, recommender_5):
        super(Linear_Hybrid, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
    def fit(self, norm=2, alpha = 0.5, beta =0.5, teta = 0.5, gamma = 0.5):

        self.alpha = alpha
        self.beta = beta
        self.teta = teta
        self.gamma = gamma
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array,items_to_compute)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array,items_to_compute)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array,items_to_compute)
        item_weights_4 = self.recommender_4._compute_item_score(user_id_array,items_to_compute)
        item_weights_5 = self.recommender_5._compute_item_score(user_id_array,items_to_compute)
        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        norm_item_weights_3 = LA.norm(item_weights_3, self.norm)
        norm_item_weights_4 = LA.norm(item_weights_4, self.norm)
        norm_item_weights_5 = LA.norm(item_weights_5, self.norm)


        
        '''
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
          if norm_item_weights_2 == 0 and self.recommender_2==EASE_R_Recommender:
            item_weights = (item_weights_1 / norm_item_weights_1)
        else:
            item_weights = (item_weights_1 / norm_item_weights_1) * self.alpha + (item_weights_2 / norm_item_weights_2) * (1-self.alpha)
        return item_weights
        '''
      
        item_weights = (item_weights_1 / norm_item_weights_1) * self.alpha + (item_weights_2 / norm_item_weights_2) * self.beta + (item_weights_3 / norm_item_weights_3) * self.teta + (item_weights_4 / norm_item_weights_4) * self.gamma + self.teta + (item_weights_5 / norm_item_weights_5) * (1-self.alpha-self.beta-self.teta-self.gamma)
        
        return item_weights

In [30]:
dataReader = DataReader()

target = dataReader.load_target()

URM = dataReader.load_augmented_binary_urm()
URM_aug,icm = dataReader.pad_with_zeros_ICMandURM(URM)

URM_train_aug, URM_validation = split_train_in_two_percentage_global_sample(URM_aug, train_percentage = 0.9)
URM_train_pow = dataReader.stackMatrixes(URM_train_aug)
UCM = dataReader.load_aug_ucm()


evaluator_validation = EvaluatorHoldout(URM_validation, [10])

>>> number of target users: 41116
Making augmented URM and ICM of the same shape...
EvaluatorHoldout: Ignoring 2857 ( 6.9%) Users that have less than 1 test interactions


In [31]:
UserKNNCF = UserKNNCFRecommender(URM_train_aug)
UserKNNCF.fit()

UserKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 41629 (100.0%), 569.60 column/sec. Elapsed time 1.22 min


In [32]:
ItemKNNCF = ItemKNNCFRecommender(URM_train_pow)
ItemKNNCF.fit()

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 27968 (100.0%), 567.72 column/sec. Elapsed time 49.26 sec


In [33]:
RP3beta_aug = RP3betaRecommender(URM_train_aug)
RP3beta_aug.fit()

RP3betaRecommender: URM Detected 3461 (12.4%) items with no interactions.
RP3betaRecommender: Similarity column 27968 (100.0%), 3570.52 column/sec. Elapsed time 7.83 sec


In [34]:
RP3beta_pow = RP3betaRecommender(URM_train_pow)
RP3beta_pow.fit(alpha=0.3648761546066018,beta=0.5058870363874656, topK=480, normalize_similarity=True)

RP3betaRecommender: Similarity column 27968 (100.0%), 1349.05 column/sec. Elapsed time 20.73 sec


In [35]:
S_SLIM = SLIMElasticNetRecommender(URM_train_pow)
S_SLIM.fit()

SLIMElasticNetRecommender: Processed 5685 (20.3%) in 5.00 min. Items per second: 18.95
SLIMElasticNetRecommender: Processed 11709 (41.9%) in 10.00 min. Items per second: 19.51
SLIMElasticNetRecommender: Processed 17742 (63.4%) in 15.00 min. Items per second: 19.71
SLIMElasticNetRecommender: Processed 23773 (85.0%) in 20.00 min. Items per second: 19.81
SLIMElasticNetRecommender: Processed 27968 (100.0%) in 21.09 min. Items per second: 22.10


In [36]:
EASE_R = EASE_R_Recommender(URM_train_aug)
EASE_R.fit()

EASE_R_Recommender: URM Detected 3461 (12.4%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 6.21 min


In [37]:
Hybrid_SSLIM_EASER = Hybrid_SSLIM_EASER(URM_train_aug, URM_train_pow, S_SLIM, EASE_R)
Hybrid_SSLIM_EASER.fit(SSLIM_weight=0.563368095251961, EASE_R_weight=0.0)

Hybrid_layer1: URM Detected 3461 (12.4%) items with no interactions.
